<a href="https://colab.research.google.com/github/IntroComputationalPhysics-UNT/connecting-to-github-classroom-nworbehc/blob/main/PeriodAndRevolutionTime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PSEUDOCODE FOR REVOLUTION TIME**
1. Import necessary modules
    - numpy, matplotlib.pyplot, solve_ivp from scipy.integrate, quad from scipy.integrate
2. Define the ODE, define the integrand
    - ode_func(statevar, t, omega_0), name it something fun
    - revtime_integrand(args)
3. Define utility functions for plotting etc
4. Define event functions
5. Solve for revolution time $T_r$ using event/solve_idp and again using quad
6. Plot the results together, compare

In [5]:
#STEP 1: Imported goods
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, quad

#STEP 2: Define the functions

#Rigid Pendulum ODE
def Brian(t, y, omega_0=1): #defining the ode function
  """
  Takes in state variable and natural frequency,
  returns angular velocity and angular acceleration
  """
  theta, ang_vel = y  #unpacking the state variable
  ang_accel = -omega_0**2 * np.sin(theta)
  return ang_vel, ang_accel

#Integrand
def Period_Integrand(theta, ang_vel_0, omega_0=1):
  """
  Returns the integrand for revolution time that we
  found in class, takes in angle, intial angular velocity,
  and natural frequency. Assumes that initial energy is
  entirely kinetic, ie pendulum starts hanging straight down
  """
  the_integrand = 1/np.sqrt(2 * (omega_0**2) * (np.cos(theta) - 1 + (ang_vel_0**2)/(2*(omega_0**2)))) #integrand we found in class
  #i didn't use coeff/denominator because i didn't like that notation
  return the_integrand

#STEP 3: Utilities

#Plotting function
def Mr_Plot(x, y, xlabel=None, ylabel=None, figsize=(3,3)):
  """
  Takes in x and y values to plot, names for the axes,
  and sets the size for the graph.
  """
  plt.figure(figsize=figsize)
  plt.plot(x, y)
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  return plt.show()

#STEP 4: Event functions

#Period event function
def Period_Event(t, y, omega0=1):
  """
  This event function finds the turning points for
  angular velocity. Useful for the case where our initial
  angular velocity isn't enough to cause the pendulum to
  spin around
  """
  theta, ang_vel = y
  return ang_vel
Period_Event.terminal = False
Period_Event.direction = -1   #2nd derivative test: only find zeros going from pos to neg

#Revolution event function
def Revolution_Event(t, y, theta_0):
  """
  This event function finds
  """

#STEP 5: Extract the period and the revolution time
def Extract_Period_ODE(n_points, eps=1e-3, omega_0=1):
  """
  This function takes in a mesh of n_points, and uses a
  tiny epsilon to help with calculating around the asymptotes
  at ±pi. Taken from example function used in class
  """
  # extract period with integral on a list of initial angles
  theta_0_list = np.linspace(eps, np.pi - eps, n_points) # list of initial angles

  period_list_ode = [] # initialize list of periods

  t_min = 0
  t_max = 100
  t_span = [t_min, t_max] # time span for solve_ivp

  for theta_0 in theta_0_list:
    y_0 = [theta_0, 0] # define state var, initial condition

    sol = solve_ivp(Brian, t_span, y_0, args=(omega_0,),
                max_step = 0.01,
                dense_output=False,
                events=ang_vel_zero_event)

    period = np.mean(np.diff(sol.t_events))
    period_list_ode.append(period)
